In [1]:
!wget dataset_ozon.tsv https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv

--2023-03-21 16:13:12--  http://dataset_ozon.tsv/
Resolving dataset_ozon.tsv (dataset_ozon.tsv)... failed: Name or service not known.
wget: unable to resolve host address ‘dataset_ozon.tsv’
--2023-03-21 16:13:12--  https://raw.githubusercontent.com/ustera/brand_ner/main/dataset_ozon.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9640627 (9.2M) [text/plain]
Saving to: ‘dataset_ozon.tsv.1’

dataset_ozon.tsv.1  100%[===================>]   9.19M  --.-KB/s    in 0.03s   

2023-03-21 16:13:12 (283 MB/s) - ‘dataset_ozon.tsv.1’ saved [9640627/9640627]

FINISHED --2023-03-21 16:13:12--
Total wall clock time: 0.2s
Downloaded: 1 files, 9.2M in 0.03s (283 MB/s)


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
import json
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin
import srsly
from re import sub
from string import punctuation
tqdm.pandas()

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
!python3 -m spacy download ru_core_news_lg

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-21 16:13:27.329936: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 16:13:27.330074: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 16:13:27.330101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [4]:
ds = pd.read_csv('/content/dataset_ozon.tsv', sep = '\t')
ds = ds.drop(columns = ['Unnamed: 0'])

In [5]:
ds

,name,brand
0,"Игровая консоль PlayStation 5, белый",PlayStation
1,Золотой Шелк Маска для волос гиалуроновая кисл...,Золотой Шелк
2,"Игровая консоль Microsoft Xbox Series X, черный",Microsoft
3,L'Oreal Paris Infaillible 24H Fresh Wear Пудра...,L'Oreal Paris
4,Maybelline New York Lifter Gloss Блеск для губ...,Maybelline New York
...,...,...
81774,"Erborian BB Family BB карандаш, светлый, 3 г",Erborian
81775,Комплект модулей сменных фильтрующих Аквафор Р...,Аквафор
81776,Набор менструальных чаш Satisfyer Feel secure ...,Satisfyer
81777,28048 Подстилка для надувных и каркасных бассе...,Intex


In [6]:
# add spaces either side of punctuation
# remove multiple spaces
# strip of spacelike characters
ds = ds.applymap(lambda string: sub(r"\s{2,}", " ", sub(f'([{punctuation}]) ?', r" \1 ", string)).strip()) 

In [7]:
ds

,name,brand
0,"Игровая консоль PlayStation 5 , белый",PlayStation
1,Золотой Шелк Маска для волос гиалуроновая кисл...,Золотой Шелк
2,"Игровая консоль Microsoft Xbox Series X , черный",Microsoft
3,L ' Oreal Paris Infaillible 24H Fresh Wear Пуд...,L ' Oreal Paris
4,Maybelline New York Lifter Gloss Блеск для губ...,Maybelline New York
...,...,...
81774,"Erborian BB Family BB карандаш , светлый , 3 г",Erborian
81775,Комплект модулей сменных фильтрующих Аквафор Р...,Аквафор
81776,Набор менструальных чаш Satisfyer Feel secure ...,Satisfyer
81777,28048 Подстилка для надувных и каркасных бассе...,Intex


In [8]:
import ru_core_news_lg
nlp = ru_core_news_lg.load()

In [9]:
def make_tags(name, brand):
  global tags
  global ind
  name_splt = nlp(name)
  brand_splt = nlp(brand)
  for i in name_splt:
    for j in brand_splt:
      # check for brand names that repeat words (e.g. M&M's)
      if not tags or tags[-1][1].idx != i.idx:
        if i.lower_ == j.lower_:
          if j.i == 0:
            # for multiple word brand names, we have to make sure what the word found is part of the brand name
            if len(brand_splt) == 1: 
              tags.append([ind, i, 'B-BRAND'])
            elif name_splt[i.i:].text.lower().startswith(brand_splt[j.i:].text.lower()):
              tags.append([ind, i, 'B-BRAND'])
            elif (j.i == len(brand_splt)-1) or (i.text not in brand_splt[j.i+1:].text.split()):
              tags.append([ind, i, 'O'])
          else: 
            if (
                name_splt[:i.i].text.lower().endswith(brand_splt[:j.i].text.lower())
                and
                name_splt[i.i:].text.lower().startswith(brand_splt[j.i:].text.lower())
              ):
              tags.append([ind, i, 'I-BRAND'])
            elif (j.i == len(brand_splt)-1) or (i.text not in brand_splt[j.i+1:].text.split()):
              tags.append([ind, i, 'O'])
    if i.lower_ not in brand.lower():
      tags.append([ind, i, 'O'])
  ind += 1
  return tags

In [10]:
tags = []
ind = 0
print(make_tags("Pure Paw Paw бальзам с ароматом клубники", "Pure Paw Paw"))

[[0, Pure, 'B-BRAND'], [0, Paw, 'I-BRAND'], [0, Paw, 'I-BRAND'], [0, бальзам, 'O'], [0, с, 'O'], [0, ароматом, 'O'], [0, клубники, 'O']]


In [11]:
tags = []
ind = 0
nlp = spacy.load('ru_core_news_lg', disable=['ner', 'parser', 'tok2vec', 'attribute_ruler', 'morphologizer'])
ds.progress_apply(lambda x: make_tags(x['name'], x['brand']), axis = 1)

100%|██████████| 81779/81779 [04:10<00:00, 326.11it/s]


0        [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
1        [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
2        [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
3        [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
4        [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
                               ...                        
81774    [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
81775    [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
81776    [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
81777    [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
81778    [[0, Игровая, O], [0, консоль, O], [0, PlaySta...
Length: 81779, dtype: object

In [12]:
ds_IOB2 = pd.DataFrame(tags, columns = ['sent_id', 'token', 'label'])

In [13]:
ds_IOB2

,sent_id,token,label
0,0,Игровая,O
1,0,консоль,O
2,0,PlayStation,B-BRAND
3,0,5,O
4,0,",",O
...,...,...,...
1073593,81777,см,O
1073594,81778,Трусы,O
1073595,81778,женские,O
1073596,81778,befree,B-BRAND


In [14]:
train, test = train_test_split(ds, test_size=0.2)
train, dev = train_test_split(train, test_size=0.2)

In [15]:
train.shape, dev.shape, test.shape

((52338, 2), (13085, 2), (16356, 2))

In [16]:
# train.to_csv('train_ozon.csv',index=False)
# test.to_csv('test_ozon.csv',index=False)
# dev.to_csv('dev_ozon.csv',index=False)

In [17]:
train

,name,brand
66784,"Сидр "" Лось и Кедр "" безалкогольный релиз 0 . ...",Лось и Кедр
2673,"Лосьон для кожи головы Charismo Deko3Pill , на...",Charismo
6075,Фигурка Funko POP ! Bobble Marvel The Falcon a...,Funko
24723,Диван - кровать Нижегородмебель и К Вилли ( 12...,Нижегородмебель и К
40072,Logitech G G203 LightSync компьютерная мышь US...,Logitech
...,...,...
70203,Вкусный жевательный мармелад из Испании Арбуз ...,Остров сладостей
48249,Трусы купальные Infinity Lingerie жен . Oira ч...,Infinity Lingerie
76966,"Внешний аккумулятор Xiaomi Mi Power Bank 2 , 1...",Xiaomi
66749,CRAZY COLOR Краситель оттеночный 68 лаймовая ц...,CRAZY COLOR


Converting to spaCy format. We need brand start and end endices in the sentence

In [19]:
brand_indices = {}
sents = ds_IOB2.groupby('sent_id')

cnt = 0
failed = []
for sent_id, sent in tqdm(sents):
  start, end = -1, -1
  text_list = sent['token'].tolist()
  labels = sent['label'].tolist()
  for n, token in enumerate(text_list):
    if labels[n][0] == 'B':
      start = token.idx
      if n == len(labels)-1 or labels[n+1][0] == 'O':
        # brand name is only one word
        end = token.idx + len(token)
    elif labels[n][0] == 'I' and (
         n == len(labels)-1 or labels[n+1][0] == 'O'
         ):
      # brand name is multiple words
      end = token.idx + len(token)
  if start != -1 and end != -1:
    if len(ds.iloc[sent_id]['brand']) == end-start:
      brand_indices[sent_id] = {'brand': ds.iloc[sent_id]['brand'], 'ind': [start, end]}
    else:
      cnt += 1
      failed.append(sent_id)
    # assert len(ds.iloc[sent_id]['brand']) == end-start, f"{sent}, \ntarget {ds.iloc[sent_id]['brand']}, ind {start}, {end}"

100%|██████████| 81779/81779 [00:20<00:00, 4030.29it/s]


In [20]:
cnt

30

In [ ]:
db = spacy.tokens.DocBin()
train_id = train.index

for idx in tqdm(train_id):
  if idx in brand_indices.keys():
    doc = nlp(train.loc[train.index == idx]['name'].tolist()[0])
    ents = []
    start, end = brand_indices[idx]['ind']
    span = doc.char_span(start, end, label='BRAND')
    ents.append(span)
    doc.ents = ents
    db.add(doc)

In [ ]:
db.to_disk('./train.spacy')

In [ ]:
db = spacy.tokens.DocBin()
test_id = test.index

for idx in tqdm(test_id):
  if idx in brand_indices.keys():
    doc = nlp(test.loc[test.index == idx]['name'].tolist()[0])
    ents = []
    start, end = brand_indices[idx]['ind']
    span = doc.char_span(start, end, label='BRAND')
    ents.append(span)
    doc.ents = ents
    db.add(doc)

In [ ]:
db.to_disk('./test.spacy')

we have a spacy-specific format for training. Now we can train our own spacy NER model to predict brands -- but first, we need custom vectors

create embeddings

In [21]:
!pip install gensim --upgrade

import gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
ds

,name,brand
0,"Игровая консоль PlayStation 5 , белый",PlayStation
1,Золотой Шелк Маска для волос гиалуроновая кисл...,Золотой Шелк
2,"Игровая консоль Microsoft Xbox Series X , черный",Microsoft
3,L ' Oreal Paris Infaillible 24H Fresh Wear Пуд...,L ' Oreal Paris
4,Maybelline New York Lifter Gloss Блеск для губ...,Maybelline New York
...,...,...
81774,"Erborian BB Family BB карандаш , светлый , 3 г",Erborian
81775,Комплект модулей сменных фильтрующих Аквафор Р...,Аквафор
81776,Набор менструальных чаш Satisfyer Feel secure ...,Satisfyer
81777,28048 Подстилка для надувных и каркасных бассе...,Intex


In [23]:
# Building sentences

titles = ds['name'].tolist()
sequences = list(map(lambda x: x.split(), titles))

In [24]:
len(sequences)

81779

In [25]:
model = gensim.models.Word2Vec(sequences, min_count=2, vector_size=200, epochs=10)

In [27]:
model.wv.most_similar('детская')

[('Кроватка', 0.808512270450592),
 ('Горка', 0.7847390174865723),
 ('KIDS', 0.7786445617675781),
 ('приставная', 0.7524115443229675),
 ('кроватка', 0.7435532808303833),
 ('50548', 0.7391736507415771),
 ('люлька', 0.7380459308624268),
 ('Babycare', 0.7362977862358093),
 ('Новомосковский', 0.7315843105316162),
 ('каталка', 0.7288681864738464)]

In [33]:
model.wv.similarity('Apple','Samsung')

0.7698821

In [34]:
model.wv.save_word2vec_format('embeddings.txt', binary=False)

In [35]:
!python -m spacy init vectors ru embeddings.txt spacy_embeddings --name title_vectors

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-21 16:24:54.264395: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 16:24:54.264517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 16:24:54.264536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [36]:
nlp = spacy.load('spacy_embeddings')

In [37]:
nlp.add_pipe('ner')
nlp.to_disk('spacy_embeddings')

In [40]:
!python -m spacy train spacy_embeddings/config.cfg --output spacy_model --paths.train train.spacy --paths.dev test.spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-21 16:37:47.120616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 16:37:47.120801: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 16:37:47.120835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [43]:
!zip -r /content/spacy_model.zip /content/spacy_model

  adding: content/spacy_model/ (stored 0%)
  adding: content/spacy_model/model-last/ (stored 0%)
  adding: content/spacy_model/model-last/meta.json (deflated 54%)
  adding: content/spacy_model/model-last/vocab/ (stored 0%)
  adding: content/spacy_model/model-last/vocab/lookups.bin (stored 0%)
  adding: content/spacy_model/model-last/vocab/vectors.cfg (stored 0%)
  adding: content/spacy_model/model-last/vocab/vectors (deflated 45%)
  adding: content/spacy_model/model-last/vocab/strings.json (deflated 86%)
  adding: content/spacy_model/model-last/vocab/key2row (stored 0%)
  adding: content/spacy_model/model-last/config.cfg (deflated 59%)
  adding: content/spacy_model/model-last/tokenizer (deflated 84%)
  adding: content/spacy_model/model-last/ner/ (stored 0%)
  adding: content/spacy_model/model-last/ner/model (deflated 8%)
  adding: content/spacy_model/model-last/ner/moves (deflated 42%)
  adding: content/spacy_model/model-last/ner/cfg (deflated 33%)
  adding: content/spacy_model/model-b

In [ ]:
model.wv.save_word2vec_format('embeddings.txt', binary=False)

In [ ]:
!python -m spacy init vectors ru embeddings.txt spacy_embeddings

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-11 15:12:15.337289: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 15:12:15.337439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 15:12:15.337464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
def load_word_vectors(model_name, word_vectors):
    import spacy
    import subprocess
    import sys
    subprocess.run([sys.executable,
                    "-m",
                    "spacy",
                    "init-model",
                    "en",
                    model_name,
                    "--vectors-loc",
                    word_vectors
                        ]
                    )
    
load_word_vectors('custom_model', 'embeddings.txt')

In [ ]:
!python -m spacy --help

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-11 14:54:36.815624: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 14:54:36.815745: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 14:54:36.815769: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
!python -m spacy init vectors ru custom_model --vectors-loc embeddings.txt

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-11 14:53:36.617447: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 14:53:36.617573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-11 14:53:36.617597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [ ]:
nlp.add_pipe('ner')
nlp.to_disk('models/spacy_embeddings')